<div align="center">
  <a target="_blank" href="https://colab.research.google.com/github/crowdcent/crowdcent-challenge/blob/main/docs/tutorials/hyperliquid-end-to-end.ipynb">
    <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
  </a>
</div>

In [ ]:
!pip install crowdcent-challenge
import crowdcent_challenge as cc
import polars as pl
from xgboost import XGBRegressor

## Build a model on CrowdCent's training data and submit

For this tutorial, you will need:
1) **CrowdCent API Key**: [register for free](https://crowdcent.com/accounts/signup/) and generate an API key from your [user profile](https://crowdcent.com/profile).

### Load API key

In [1]:
CROWDCENT_API_KEY = "API_KEY_HERE"

### Initialize the client

In [2]:
client = cc.ChallengeClient(
    challenge_slug="hyperliquid-ranking",
    api_key=CROWDCENT_API_KEY,
)

2025-06-09 16:19:54,459 - INFO - ChallengeClient initialized for 'hyperliquid-ranking' at URL: https://crowdcent.com/api


### Get CrowdCent's training data

In [3]:
client.download_training_dataset(version="latest", dest_path="training_data.parquet")

training_data = pl.read_parquet("training_data.parquet")
training_data.head()

2025-06-09 16:19:54,715 - INFO - Downloading training data for challenge 'hyperliquid-ranking' v1.0 to training_data.parquet
2025-06-09 16:19:55,971 - INFO - Successfully downloaded training data to training_data.parquet


id,eodhd_id,date,feature_16_lag15,feature_13_lag15,feature_14_lag15,feature_15_lag15,feature_8_lag15,feature_5_lag15,feature_6_lag15,feature_7_lag15,feature_12_lag15,feature_9_lag15,feature_10_lag15,feature_11_lag15,feature_4_lag15,feature_1_lag15,feature_2_lag15,feature_3_lag15,feature_20_lag15,feature_17_lag15,feature_18_lag15,feature_19_lag15,feature_16_lag10,feature_13_lag10,feature_14_lag10,feature_15_lag10,feature_8_lag10,feature_5_lag10,feature_6_lag10,feature_7_lag10,feature_12_lag10,feature_9_lag10,feature_10_lag10,feature_11_lag10,feature_4_lag10,feature_1_lag10,…,feature_5_lag5,feature_6_lag5,feature_7_lag5,feature_12_lag5,feature_9_lag5,feature_10_lag5,feature_11_lag5,feature_4_lag5,feature_1_lag5,feature_2_lag5,feature_3_lag5,feature_20_lag5,feature_17_lag5,feature_18_lag5,feature_19_lag5,feature_16_lag0,feature_13_lag0,feature_14_lag0,feature_15_lag0,feature_8_lag0,feature_5_lag0,feature_6_lag0,feature_7_lag0,feature_12_lag0,feature_9_lag0,feature_10_lag0,feature_11_lag0,feature_4_lag0,feature_1_lag0,feature_2_lag0,feature_3_lag0,feature_20_lag0,feature_17_lag0,feature_18_lag0,feature_19_lag0,target_10d,target_30d
str,str,datetime[μs],f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""BRETT""","""BRETT29743-USD.CC""",2024-05-06 00:00:00,0.55721,0.540086,0.512754,0.508386,0.692426,0.55288,0.549131,0.533897,0.736748,0.617263,0.603953,0.623957,0.450295,0.479222,0.519303,0.528181,0.527312,0.525878,0.497428,0.518564,0.239416,0.398313,0.451205,0.487323,0.118248,0.405337,0.448927,0.492024,0.086131,0.41144,0.485798,0.550351,0.236496,0.343396,…,0.177372,0.365126,0.493638,0.351825,0.218978,0.418121,0.546089,0.268613,0.252555,0.365888,0.495271,0.420438,0.551095,0.538486,0.542948,0.233333,0.249513,0.323913,0.447919,0.289855,0.263176,0.334256,0.435102,0.428986,0.390405,0.400922,0.508626,0.272464,0.270538,0.306967,0.431848,0.401449,0.410944,0.507738,0.507471,0.507246,0.985507
"""BRETT""","""BRETT29743-USD.CC""",2024-05-07 00:00:00,0.456339,0.471133,0.514562,0.508886,0.54709,0.519471,0.515427,0.532992,0.613331,0.547406,0.606655,0.627392,0.563407,0.461703,0.512452,0.548603,0.517968,0.518984,0.491352,0.514658,0.233577,0.344958,0.446484,0.48631,0.116788,0.331939,0.402621,0.492332,0.056934,0.335133,0.450072,0.55014,0.110949,0.337178,…,0.175161,0.347316,0.470913,0.351814,0.204374,0.37589,0.527665,0.230646,0.170798,0.31625,0.470839,0.374209,0.602433,0.560709,0.53946,0.272464,0.269793,0.307375,0.428926,0.407246,0.32039,0.326165,0.426455,0.484058,0.417936,0.376534,0.491264,0.365217,0.297932,0.317555,0.420723,0.281159,0.327684,0.500999,0.487154,0.869565,0.985507
"""BRETT""","""BRETT29743-USD.CC""",2024-05-08 00:00:00,0.566488,0.453614,0.507711,0.529308,0.383759,0.434102,0.471631,0.518237,0.448497,0.491656,0.563988,0.615796,0.37807,0.391257,0.465007,0.524881,0.46842,0.55421,0.480155,0.521811,0.106569,0.336529,0.407477,0.490267,0.287591,0.335675,0.417691,0.514816,0.230657,0.339577,0.45308,0.574943,0.291971,0.33502,…,0.262028,0.348065,0.471507,0.332974,0.281815,0.386736,0.525698,0.239405,0.265688,0.328473,0.472002,0.498498,0.616402,0.585306,0.538537,0.365217,0.297916,0.317222,0.417801,0.255072,0.245769,0.290722,0.407634,0.314493,0.323733,0.331655,0.468477,0.171014,0.20521,0.270115,0.396829,0.205797,0.352147,0.476755,0.47601,0.913043,0.985507
"""BRETT""","""BRETT29743-USD.CC""",2024-05-09 00:00:00,0.382578,0.383157,0.46101,0.505768,0.401234,0.42504,0.441107,0.496157,0.321855,0.413705,0.524751,0.593716,0.40717,0.410203,0.472939,0.505173,0.590994,0.579914,0.497107,0.513647,0.286131,0.334355,0.427468,0.513196,0.268613,0.334924,0.391343,0.490949,0.337226,0.329541,0.440073,0.566184,0.254015,0.330593,…,0.261943,0.343492,0.450375,0.403047,0.370136,0.391921,0.52686,0.242272,0.248143,0.329173,0.449071,0.554988,0.581

### Train a model on the training data

In [4]:
xgb_regressor = XGBRegressor()
feature_names = [col for col in training_data.columns if col.startswith("feature")]

xgb_regressor.fit(
    training_data[feature_names],
    training_data[["target_10d", "target_30d"]],
)

,objective,'reg:squarederror'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


### Get CrowdCent's latest inference data

In [5]:
client.download_inference_data("current", "inference_data.parquet")

inference_data = pl.read_parquet("inference_data.parquet")
inference_data.head()

2025-06-09 16:19:58,205 - INFO - Downloading inference data for challenge 'hyperliquid-ranking' current to inference_data.parquet
2025-06-09 16:19:58,441 - INFO - Successfully downloaded inference data to inference_data.parquet
2025-06-09 16:19:58,442 - INFO - Successfully downloaded inference data after 1 attempt(s) to inference_data.parquet


id,eodhd_id,date,feature_16_lag15,feature_13_lag15,feature_14_lag15,feature_15_lag15,feature_8_lag15,feature_5_lag15,feature_6_lag15,feature_7_lag15,feature_12_lag15,feature_9_lag15,feature_10_lag15,feature_11_lag15,feature_4_lag15,feature_1_lag15,feature_2_lag15,feature_3_lag15,feature_20_lag15,feature_17_lag15,feature_18_lag15,feature_19_lag15,feature_16_lag10,feature_13_lag10,feature_14_lag10,feature_15_lag10,feature_8_lag10,feature_5_lag10,feature_6_lag10,feature_7_lag10,feature_12_lag10,feature_9_lag10,feature_10_lag10,feature_11_lag10,feature_4_lag10,feature_1_lag10,…,feature_15_lag5,feature_8_lag5,feature_5_lag5,feature_6_lag5,feature_7_lag5,feature_12_lag5,feature_9_lag5,feature_10_lag5,feature_11_lag5,feature_4_lag5,feature_1_lag5,feature_2_lag5,feature_3_lag5,feature_20_lag5,feature_17_lag5,feature_18_lag5,feature_19_lag5,feature_16_lag0,feature_13_lag0,feature_14_lag0,feature_15_lag0,feature_8_lag0,feature_5_lag0,feature_6_lag0,feature_7_lag0,feature_12_lag0,feature_9_lag0,feature_10_lag0,feature_11_lag0,feature_4_lag0,feature_1_lag0,feature_2_lag0,feature_3_lag0,feature_20_lag0,feature_17_lag0,feature_18_lag0,feature_19_lag0
str,str,datetime[μs],f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""PYTH""","""PYTH-USD.CC""",2025-06-08 00:00:00,0.356213,0.313268,0.388676,0.430942,0.366864,0.275218,0.366181,0.421273,0.466272,0.318431,0.404669,0.424983,0.499408,0.345087,0.411836,0.434626,0.405917,0.415427,0.435123,0.465662,0.607101,0.481657,0.423885,0.440786,0.626036,0.49645,0.397689,0.440551,0.521893,0.494083,0.41707,0.421781,0.491124,0.495266,…,0.436416,0.385799,0.505917,0.390568,0.420564,0.513609,0.517751,0.418091,0.437646,0.544379,0.517751,0.431419,0.434151,0.433136,0.44497,0.430199,0.463895,0.622485,0.527811,0.504734,0.448413,0.501775,0.443787,0.470118,0.425605,0.571598,0.542604,0.518343,0.454728,0.60355,0.573964,0.534615,0.459444,0.435503,0.43432,0.43284,0.463965
"""RSR""","""RSR-USD.CC""",2025-06-08 00:00:00,0.578698,0.490064,0.451651,0.497864,0.55503,0.473577,0.437025,0.484949,0.486391,0.492466,0.42143,0.46844,0.507692,0.494879,0.432078,0.489316,0.462722,0.458376,0.504701,0.482021,0.357396,0.468047,0.409976,0.470975,0.404734,0.479882,0.418329,0.479957,0.304142,0.395266,0.3809,0.444279,0.330178,0.418935,…,0.465218,0.423669,0.414201,0.443889,0.469804,0.427219,0.36568,0.429073,0.447165,0.504142,0.41716,0.456019,0.468522,0.52071,0.532544,0.49546,0.499041,0.547929,0.476331,0.472189,0.457354,0.423669,0.423669,0.451775,0.43963,0.55503,0.491124,0.443195,0.444556,0.439053,0.471598,0.445266,0.449458,0.388166,0.454438,0.478994,0.49214
"""INJ""","""INJ-USD.CC""",2025-06-08 00:00:00,0.605917,0.57736,0.588146,0.576009,0.718343,0.637651,0.606953,0.567946,0.731361,0.573006,0.607233,0.56306,0.725444,0.588993,0.61576,0.585962,0.626036,0.606879,0.565018,0.553379,0.75503,0.680473,0.621783,0.608503,0.622485,0.670414,0.609864,0.577455,0.652071,0.691716,0.603985,0.585323,0.597633,0.661538,…,0.564593,0.480473,0.551479,0.594565,0.556858,0.327811,0.489941,0.531474,0.553963,0.418935,0.508284,0.548639,0.561156,0.502959,0.492899,0.549889,0.534348,0.740828,0.517751,0.599112,0.591233,0.759763,0.620118,0.645266,0.601226,0.687574,0.507692,0.599704,0.58509,0.784615,0.601775,0.631657,0.608329,0.569231,0.536095,0.545266,0.539689
"""ZEN""","""ZEN-USD.CC""",2025-06-08 00:00:00,0.530178,0.541846,0.524401,0.509716,0.572781,0.522647,0.535058,0.514253,0.55858,0.481942,0.474025,0.474323,0.553846,0.528716,0.516441,0.505751,0.501775,0.492466,0.515593,0.498992,0.523077,0.526627,0.513002,0.484648,0.511243,0.542012,0.519194,0.494608,0.480473,0.519527,0.467637,0.444861,0.51716,0.535503,…,0.528473,0.665089,0.588166,0.555406,0.522402,0.816568,0.648521,0.565231,0.498992,0.738462,0.627811,0.578263,0.529029,0.459172,0.490533,0.49149

### Make predictions on the inference data

In [6]:
preds = xgb_regressor.predict(inference_data[feature_names])
pred_df = pl.from_numpy(preds, ["pred_10d", "pred_30d"])

### Submit to the `hyperliquid-ranking` challenge on CrowdCent

In [7]:
pred_df = pred_df.with_columns(inference_data["id"]).select(
    ["id", "pred_10d", "pred_30d"]
)

with pl.Config(tbl_rows=20):
    display(pred_df.sort("pred_30d", descending=True))

id,pred_10d,pred_30d
str,f32,f32
"""MERL""",0.522567,0.726309
"""APT""",0.636483,0.693763
"""INJ""",0.719315,0.64265
"""RUNE""",0.649341,0.63357
"""IP""",0.602131,0.63068
"""AVAX""",0.648095,0.623901
"""EIGEN""",0.638704,0.622778
"""BTC""",0.636244,0.616681
"""XLM""",0.612878,0.609954


In [8]:
# directly submit a dataframe
for slot in [1, 2]:
    client.submit_predictions(df=pred_df, slot=slot)

2025-06-09 16:19:58,549 - INFO - Wrote DataFrame to temporary file: submission.parquet
2025-06-09 16:19:58,550 - INFO - Submitting predictions from submission.parquet to challenge 'hyperliquid-ranking' (Slot: 1)
2025-06-09 16:19:59,113 - INFO - Successfully submitted predictions to challenge 'hyperliquid-ranking'
2025-06-09 16:19:59,117 - INFO - Wrote DataFrame to temporary file: submission.parquet
2025-06-09 16:19:59,118 - INFO - Submitting predictions from submission.parquet to challenge 'hyperliquid-ranking' (Slot: 2)
2025-06-09 16:19:59,518 - INFO - Successfully submitted predictions to challenge 'hyperliquid-ranking'
